<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/L_test/email-subject/model-tuning/AESLC_tuning_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 81 (delta 27), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (81/81), 7.73 MiB | 4.79 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [2]:
!pip install transformers
!pip install evaluate
!pip install rouge-score # Installing rouge-score library (https://pypi.org/project/rouge-score/)

# !pip install accelerate -U

!pip install transformers[torch]
!pip install sacrebleu

!pip install datasets nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [3]:
# Importing Libraries


import os
import re
import pandas as pd # Data Handling
import tensorflow as tf
import numpy as np
import torch
import time
import seaborn as sns

from datasets import load_metric
from datasets import Dataset
import evaluate

from google.colab import files
import shutil

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration      # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments         # These will help us to fine-tune our model
from transformers import pipeline                                         # Pipeline
from transformers import DataCollatorForSeq2Seq                           # DataCollator to batch the data

from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [66]:
!pip install nltk
import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

# **Load Dataset**

In [5]:
dataset_df = pd.read_csv('/content/qM-AI-L/email-subject/model-tuning/train_emails.csv')

In [6]:
dataset_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,allen-p_inbox_20,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement,NaN,NaN,NaN,65,2,gregphillip attached is the grande communicati...
1,allen-p_inbox_28,Phillip & Keith Attached is the first draw re...,Bishops Corner,NaN,NaN,NaN,145,2,phillip keith attached is the first draw reque...
2,allen-p_inbox_63,Your Internet Banking accounts are now setup a...,Internet Banking,NaN,NaN,NaN,250,2,your internet banking accounts are now setup a...
3,allen-p_inbox_64,To our IBS Customers that are still hanging in...,Internet Banking,NaN,NaN,NaN,458,2,to our ibs customers that are still hanging in...
4,allen-p_inbox_65,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories,NaN,NaN,NaN,68,4,phillip good morning i hope you had a wonderfu...
...,...,...,...,...,...,...,...,...,...
14431,zufferli-j_inbox_43,This email is acknowledgement from the Power P...,Power Pool,NaN,NaN,NaN,227,2,this email is acknowledgement from the power p...
14432,zufferli-j_inbox_44,This email is acknowledgement from the Power P...,Power Pool of Alberta,NaN,NaN,NaN,277,4,this email is acknowledgement from the power p...
14433,zufferli-j_inbox_46,"John, Further to the voice message that I lef...",Enron Security,NaN,NaN,NaN,148,2,john further to the voice message that i left ...
14434,zufferli-j_inbox_8,Make sure that all curves are downloaded by th...,Simulation Curves,NaN,NaN,NaN,66,2,make sure that all curves are downloaded by th...


# **Test Train Split**

In [7]:
train_df, test_df = train_test_split(dataset_df, test_size=0.2, random_state=42)

In [8]:
train_df.shape, test_df.shape

((11548, 9), (2888, 9))

In [9]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [10]:
train_dataset, test_dataset

(Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 11548
 }),
 Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 2888
 }))

**Create a subset of Dataset**

In [11]:
#create a subset of dataset
train_dataset = train_dataset.filter(lambda example, index: index % 5 == 0, with_indices=True)
test_dataset = test_dataset.filter(lambda example, index: index % 5 == 0, with_indices=True)

Filter:   0%|          | 0/11548 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2888 [00:00<?, ? examples/s]

In [12]:
train_dataset.shape, test_dataset.shape

((2310, 10), (578, 10))

In [13]:
train_dataset['cleaned_emails'][0]

'met early this morning with jack steward pres. of cma and keith mccrea lawyer for cma to follow up on the meeting that steve and i had with them regarding the proposal ken lay was pushing last week they like taking tx out of the mou they like getting dwr out of the power purchase business and they think that edisons mou needs a little haircut. mccrea pushed back initially on corenoncore why should it be mandatory to go the market but after much discussion they agreed that our idea for corenoncore has a lot of merit. our idea is that 1 the corenoncore would take effect in 1824 months when the markets settle and prices come down 2 everyone core and noncore should pay for dwrs past purchases since everyone consumed it 3 core customers should take iou generation qfs and dwr contracts which eliminates the cores net short position and 4 noncore would go to market and would not have to pay for any of dwrs going forward costs since core gets them in exchange for getting all the lowcost iou ge

**Load the pre-trained FLAN-T5 models and its tokenizer directly from HuggingFace. FLAN-T5 was released in the paper Scaling Instruction-Finetuned Language Models - it is an enhanced version of T5 that has been finetuned in a mixture of tasks. Setting torch_dtype=torch.bfloat16 specifies the memory type to be used by this model.**

In [14]:
# Function to get size of models
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\nPercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



**Load Flan T5 Base Model**

In [15]:

flan_t5_base_original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
flan_t5_base_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

print("For Model : FLAN-T5-Base \n",print_number_of_trainable_model_parameters(flan_t5_base_original_model))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

For Model : FLAN-T5-Base 
 Trainable model parameters: 247577856
All model parameters: 247577856
Percentage of trainable model parameters: 100.00%


**Load Flan T5 Small Model**

In [16]:
flan_t5_small_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
# flan_t5_small_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

print("For Model : Flan-t5-small \n",print_number_of_trainable_model_parameters(flan_t5_small_model))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

For Model : Flan-t5-small 
 Trainable model parameters: 76961152
All model parameters: 76961152
Percentage of trainable model parameters: 100.00%


In [17]:
# flan-t5-base vs. flan-t5-small parameters
247577856-76961152

170616704

# **Test the Model with Zero Shot Inferencing**

**Using a pre-trained model from Hugging Face/sources - FLAN-T5-Base .**

In [18]:
index = 10

email = train_dataset['cleaned_emails'][index]
subject = train_dataset['subject'][index]

prompt = f"""
Generate the subject line for the following email.

Email:
{email}

Subject:
"""

inputs = flan_t5_base_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(flan_t5_base_original_model.device) for k, v in inputs.items()}


output = flan_t5_base_tokenizer.decode(
    flan_t5_base_original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5 MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate the subject line for the following email.

Email:
many of you have asked about procedures to evaluate employee performance. we are finalizing a performance process designed to track and provide feedback on individual accomplishments for the benefit of each employee the company and the unsecured creditors committee. a very simple and straightforward online performance assessment system pas will allow us to automate parts of the process. pas will open next week on monday may 13 and remain open through friday may 24 2002. as an employee of a debtor company you should access the system and complete your self assessment by friday may 17 2002. your supervisor will review and approve submitted accomplishments and schedule a facetoface meeting to discuss your contributions offer feedback set priorities for the upcoming period discuss challenges and potential development o

**Using a pre-trained model from Hugging Face etc- FLAN-T5-SMALL .**

In [19]:
inputs = flan_t5_base_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# inputs = {k: v.to(device) for k, v in inputs.items()}


output = flan_t5_base_tokenizer.decode(
    flan_t5_small_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5-SMALL MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate the subject line for the following email.

Email:
many of you have asked about procedures to evaluate employee performance. we are finalizing a performance process designed to track and provide feedback on individual accomplishments for the benefit of each employee the company and the unsecured creditors committee. a very simple and straightforward online performance assessment system pas will allow us to automate parts of the process. pas will open next week on monday may 13 and remain open through friday may 24 2002. as an employee of a debtor company you should access the system and complete your self assessment by friday may 17 2002. your supervisor will review and approve submitted accomplishments and schedule a facetoface meeting to discuss your contributions offer feedback set priorities for the upcoming period discuss challenges and potential development o

**By testing with various models with the zero shot inferencing, we can see that the model struggles to extract the same subject line compared to the baseline subject, but it does pull out some important information from the email which indicates the models can be fine-tuned to the task at hand.**

# **Performing Full Fine-Tuning using FLAN-T5 BASE MODEL**

In [20]:
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

# Define special tokens
special_tokens = {
    "additional_special_tokens": ["<EMAIL>", "</EMAIL>"]
}

# Add special tokens to the tokenizer
tokenizer.add_special_tokens(special_tokens)


# Load the model
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')

# Resize the model's embeddings to accommodate the new tokens
model.resize_token_embeddings(len(tokenizer))



def tokenize_function(examples):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    inputs = [start_prompt + email + end_prompt for email in examples['body']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding='max_length')

    labels = tokenizer(examples['subject'], max_length=15, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2310 [00:00<?, ? examples/s]

In [21]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2310
})

In [22]:
# Split the dataset into train and test
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [23]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.479700,1.070827
2,1.061100,1.046962
3,1.133300,1.056319


TrainOutput(global_step=1386, training_loss=3.6705537836617035, metrics={'train_runtime': 1940.8199, 'train_samples_per_second': 2.857, 'train_steps_per_second': 0.714, 'total_flos': 7591909367218176.0, 'train_loss': 3.6705537836617035, 'epoch': 3.0})

In [25]:
# Save the fine-tuned model and tokenizer
save_directory = "./finetuned_flan_t5_base"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Zip the saved model files
zip_filename = "finetuned_flan_t5_base.zip"
shutil.make_archive(base_name="finetuned_flan_t5_base", format="zip", root_dir=save_directory)

#Download the zipped checkpoint
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Evaluate the Flan T5-Base Model Quantitatively (with ROUGE Metric)**
The ROUGE metric helps quantify the validity of subject lines produced by models. It compares subjects to a "Annoted baseline" subject which is usually created by a human. While not perfect, it does indicate the overall increase in subject line generatiion effectiveness that we have accomplished by fine-tuning.

**Load the Saved Finetuned Model and Tokenizer**

In [28]:
# Unzip model checkpoint as it is zipped
# !unzip finetuned_flan_t5_base.zip -d ./finetuned_flan_t5_base



model_path = "./finetuned_flan_t5_base"
finetuned_tokenizer = T5Tokenizer.from_pretrained(model_path)
finetuned_model = T5ForConditionalGeneration.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [58]:
emails_for_score_df = train_df.sample(n=10, random_state=22)
emails_for_score_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
6042,kuykendall-t_inbox_72,"It's that time of year again, wanted to get ev...",Christmas Lunch,NaN,NaN,NaN,95,2,its that time of year again wanted to get ever...
5456,jones-t_sent_907,You have been getting the excel spreadsheets o...,Counterparty Approvals,NaN,NaN,NaN,145,2,you have been getting the excel spreadsheets o...
67,bass-e_inbox_10,Hi.\nThis is the qmail-send program at shutter...,failure notice,NaN,NaN,NaN,51,2,hi. this is the qmailsend program at shutterfl...
7892,mcconnell-m_sent_45,"Vance, I've been attempting to call you .\nYou...",Economist,NaN,NaN,NaN,56,1,vance ive been attempting to call you . your v...
1354,campbell-l_sent_143,"John, I just spoke with Dan Walker, Vice Presi...",Rocky Mountain Elk Foundation Status,NaN,NaN,NaN,120,5,john i just spoke with dan walker vice preside...
11952,skilling-j_inbox_1360,"Dear Brothers, It was good to see everyone mak...",[smu-betas] Dent pisses on Bud Wilkinson's grave,NaN,NaN,NaN,212,7,dear brothers it was good to see everyone make...
1117,campbell-l_inbox_1134,"Introducing GROW LEAN 15 As seen on NBC, CBS,...",Body Fat Loss-No hunger pains!,NaN,NaN,NaN,107,5,introducing grow lean 15 as seen on nbc cbs cn...
14177,wolfe-j_inbox_187,We are in need of the following information: ...,Important Information Needed!!!!!!!!!,NaN,NaN,NaN,34,3,we are in need of the following information em...
7148,mann-k_sent_193,Here's another draft: Note in particular ch...,CRM Rev 2,NaN,NaN,NaN,104,3,heres another draft note in particular changes...
9721,sager-e_inbox_456,The following reports have been waiting for yo...,Expense Reports Awaiting Your Approval,NaN,NaN,NaN,30,5,the following reports have been waiting for yo...


In [59]:
emails = emails_for_score_df['cleaned_emails']
original_subjects = emails_for_score_df['subject']
original_subjects


6042                                      Christmas Lunch
5456                               Counterparty Approvals
67                                         failure notice
7892                                            Economist
1354                 Rocky Mountain Elk Foundation Status
11952    [smu-betas] Dent pisses on Bud Wilkinson's grave
1117                       Body Fat Loss-No hunger pains!
14177               Important Information Needed!!!!!!!!!
7148                                            CRM Rev 2
9721               Expense Reports Awaiting Your Approval
Name: subject, dtype: object

In [60]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 462
})

In [63]:
original_subjects = []
flan_T5_subjects = []
finetuned_flan_T5_subjects = []


for index, row in emails_for_score_df.iterrows():
    email = row['body']
    original_subject = row['subject']
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """
    original_model_input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    finetuned_model_input_ids = finetuned_tokenizer(prompt, return_tensors="pt").input_ids


    original_model_outputs = flan_t5_base_original_model.generate(input_ids=original_model_input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5_subjects.append(original_model_text_output)

    instruct_model_outputs = finetuned_model.generate(input_ids=finetuned_model_input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = finetuned_tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5_subjects.append(instruct_model_text_output)

    original_subjects.append(original_subject)

zipped_subjects = list(zip(original_subjects, flan_T5_subjects, finetuned_flan_T5_subjects))

Flan_T5_df = pd.DataFrame(zipped_subjects, columns = ['original_subjects', 'T5_subjects', 'finetuned_flan_T5_subjects'])
Flan_T5_df

,original_subjects,T5_subjects,finetuned_flan_T5_subjects
0,Christmas Lunch,Christmas Lunch,Christmas Lunch
1,Counterparty Approvals,Counterparty Approvals,Counterparty Approvals
2,failure notice,qmail-send error,qmail-send error
3,Economist,Call,Call
4,Rocky Mountain Elk Foundation Status,Rocky Mountain Elk Foundation,Rocky Mountain Elk Foundation
5,[smu-betas] Dent pisses on Bud Wilkinson's grave,Dent's book,Dent's book
6,Body Fat Loss-No hunger pains!,GROW LEAN 15,GROW LEAN 15
7,Important Information Needed!!!!!!!!!,Information Needed,Information Needed
8,CRM Rev 2,Revised draft of Enron Corp.,Revised version of the CSA
9,Expense Reports Awaiting Your Approval,Expense Reports Awaiting Your Approval,Expense Reports Awaiting Your Approval


In [64]:

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

print('FLAN T5 Base Model:')
print(original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL:')
print(finetuned_model_results)

FLAN T5 Base Model:
{'rouge1': 0.5022222222222221, 'rouge2': 0.4523809523809524, 'rougeL': 0.5, 'rougeLsum': 0.5}

FINETUNED FLAN T5 BASE MODEL:
{'rouge1': 0.5022222222222221, 'rouge2': 0.4523809523809524, 'rougeL': 0.5, 'rougeLsum': 0.5}


In [82]:
!pip install nltk

import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Download the 'punkt' resource
nltk.download('punkt')

nltk.download('wordnet') # Download the WordNet corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [83]:

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu

Flan_T5_df['rougeL'], Flan_T5_df['meteor'], Flan_T5_df['sacrebleu'] = zip(*Flan_T5_df.apply(
    lambda row: evaluate_metrics(row['original_subjects'], row['T5_subjects']), axis=1))

# Calculate average scores
average_rougeL = Flan_T5_df['rougeL'].mean()
average_meteor = Flan_T5_df['meteor'].mean()
average_sacrebleu = Flan_T5_df['sacrebleu'].mean()

print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')

Average ROUGE-L: 0.5022
Average METEOR: 0.3959
Average SACREBLEU: 17.7880


In [53]:
#T5 Flan Base
# Load the ROUGE metric
rouge_metric = load_metric('rouge')

def calculate_rouge_scores(predictions, references):
    rouge_metric.add_batch(predictions=predictions, references=references)
    result = rouge_metric.compute()
    return {
        'rouge1': result['rouge1'].mid.fmeasure * 100,
        'rouge2': result['rouge2'].mid.fmeasure * 100,
        'rougeL': result['rougeL'].mid.fmeasure * 100
    }

# Generate predictions and calculate ROUGE scores
predictions = []
references = []

for example in test_dataset:
    input_ids = example['input_ids']
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension
    outputs = model.generate(input_ids)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    ref = tokenizer.decode(example['labels'], skip_special_tokens=True)

    predictions.append(pred)
    references.append(ref)

# Calculate and print ROUGE scores for each test sample
rouge_scores = []
for pred, ref in zip(predictions, references):
    score = calculate_rouge_scores([pred], [ref])
    rouge_scores.append(score)
    print(f"Prediction: {pred}\nReference: {ref}\nROUGE Scores: {score}\n")

# Convert to DataFrame for better visualization
rouge_df = pd.DataFrame(rouge_scores)
rouge_df

The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


KeyboardInterrupt: 

In [ ]:
flan_T5_subjects = []
finetuned_flan_T5_subjects = []

for _, email in enumerate(emails_for_score):
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """
    # Tokenize the input and move to the same device as the model
    inputs = tokenizer(prompt, return_tensors='pt').to(flan_T5_small_model_finetuned.device) # Move inputs to the same device as the fine-tuned model

    # Add the decoder_start_token_id to the generation configuration
    generation_config = GenerationConfig(max_new_tokens=200,
        decoder_start_token_id=tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id)



    # Generate subject using the original model
    # Make sure flan_t5_small_model is on the same device as flan_T5_small_model_finetuned
    flan_t5_small_model = model.to(flan_T5_small_model_finetuned.device)
    original_model_outputs = flan_t5_small_model.generate(**inputs, generation_config=generation_config)
    original_model_subject_output = flan_t5_small_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5small_subjects.append(original_model_subject_output)

    # Generate subject using the finetuned model
    trained_flan_t5_outputs = flan_T5_small_model_finetuned.generate(**inputs, generation_config=generation_config)
    instruct_model_text_output = flan_T5_small_tokenizer.decode(trained_flan_t5_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5small_subjects.append(instruct_model_text_output)

zipped_summaries = list(zip(original_subjects, flan_T5small_subjects, finetuned_flan_T5small_subjects))


In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [ ]:
small_test_df = test_df.sample(n=10, random_state=42)

In [ ]:
small_test_df

In [ ]:
def generate_subject_line(email_body):
    inputs = tokenizer("Generate the subject line for the following email.: " + email_body, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # Access input_ids as a key in the dictionary
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
small_test_df['generated_subject_line'] = small_test_df['cleaned_emails'].apply(generate_subject_line)
small_test_df

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # Tokenize the input for METEOR
    true_subject_tokens = nltk.word_tokenize(true_subject)
    generated_subject_tokens = nltk.word_tokenize(generated_subject)

    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject_tokens], generated_subject_tokens) # Pass tokenized input

    # SACREBLEU - Use import sacrebleu directly
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu_score # Return the score, not the module

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject'], row['generated_subject_line']), axis=1))

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
end_prompt = '\nSubject:\n'
start_prompt + email + end_prompt
print(start_prompt)
print(email)
print(end_prompt)

In [ ]:
# Tokenization for FLAN-T5 model
def flan_t5_base_tokenize_function(example):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    prompt = [start_prompt + email + end_prompt for email in example["body"]]
    example['input_ids'] = flan_t5_base_tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = flan_t5_base_tokenizer(example["subject"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_train_datasets = train_dataset.map(flan_t5_base_tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(flan_t5_base_tokenize_function, batched=True)


In [ ]:
# tokenized_train_datasets = tokenized_train_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
# tokenized_test_datasets = tokenized_test_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [ ]:
tokenized_train_datasets, tokenized_test_datasets

**FLAN-T5 BASE - Fine-Tune the Model with the Preprocessed Tokenised Dataset utiliing the built-in Hugging Face Trainer class. Then compare it with reference to the original model.**

flan_t5_base_original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
flan_t5_base_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

In [ ]:


tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

def tokenize_function(examples):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    inputs = [start_prompt + email + end_prompt for email in examples['email_body']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    labels = tokenizer(examples['subject'], max_length=50, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)


In [ ]:
# # Tokenization
# def preprocess_data(examples):
#     inputs = ["generate subject line: " + email for email in examples['cleaned_emails']]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['subject'], max_length=128, truncation=True, padding="max_length")

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# train_dataset = train_dataset.map(preprocess_data, batched=True)
# test_dataset = test_dataset.map(preprocess_data, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=flan_t5_base_original_model ,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
)

# Train the model
trainer.train()



FLAN-T5 SMALL - Fine-Tune the Model with the Preprocessed Tokenised Dataset utiliing the built-in Hugging Face Trainer class. Then compare it with reference to the original model.

flan_t5_small_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
flan_t5_small_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [ ]:
# Tokenization for FLAN-T5 SMALL model
def flan_t5_small_tokenize_function(example):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    prompt = [start_prompt + email + end_prompt for email in example["cleaned_emails"]]
    example['input_ids'] = flan_t5_small_tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = flan_t5_small_tokenizer(example["subject"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_train_datasets = train_dataset.map(flan_t5_small_tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(flan_t5_small_tokenize_function, batched=True)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=flan_t5_small_model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
)

# Train the model
trainer.train()


In [ ]:
# Save the model checkpoint
flan_t5_small_model.save_pretrained('./results/Flan_T5_small_model')
flan_t5_small_tokenizer.save_pretrained('./results/Flan_T5_small_model')

# Define and save the generation configuration
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

generation_config.save_pretrained('./results/Flan_T5_small_model')

In [ ]:
AutoModelForSeq2SeqLM
AutoTokenizer

In [ ]:
# Load the trained Flan T5 Small fine tuned model
flan_T5_small_model_directory = '/content/results/Flan_T5_small_model'
flan_T5_small_tokenizer = AutoTokenizer.from_pretrained(flan_T5_small_model_directory)
flan_T5_small_model_finetuned = AutoModelForSeq2SeqLM.from_pretrained(flan_T5_small_model_directory)

In [ ]:
emails_for_score = tokenized_test_datasets[0:10]['cleaned_emails']
original_subjects = tokenized_test_datasets[0:10]['subject']

In [ ]:
flan_T5small_subjects = []
finetuned_flan_T5small_subjects = []

for _, cleaned_emails in enumerate(emails_for_score):
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """
    # Tokenize the input and move to the same device as the model
    inputs = flan_t5_small_tokenizer(prompt, return_tensors='pt').to(flan_T5_small_model_finetuned.device) # Move inputs to the same device as the fine-tuned model

    # Add the decoder_start_token_id to the generation configuration
    generation_config = GenerationConfig(max_new_tokens=200,
        decoder_start_token_id=flan_T5_small_tokenizer.bos_token_id if flan_T5_small_tokenizer.bos_token_id is not None else flan_T5_small_tokenizer.eos_token_id
    )



    # Generate subject using the original model
    # Make sure flan_t5_small_model is on the same device as flan_T5_small_model_finetuned
    flan_t5_small_model = flan_t5_small_model.to(flan_T5_small_model_finetuned.device)
    original_model_outputs = flan_t5_small_model.generate(**inputs, generation_config=generation_config)
    original_model_subject_output = flan_t5_small_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5small_subjects.append(original_model_subject_output)

    # Generate subject using the finetuned model
    trained_flan_t5_outputs = flan_T5_small_model_finetuned.generate(**inputs, generation_config=generation_config)
    instruct_model_text_output = flan_T5_small_tokenizer.decode(trained_flan_t5_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5small_subjects.append(instruct_model_text_output)

zipped_summaries = list(zip(original_subjects, flan_T5small_subjects, finetuned_flan_T5small_subjects))

Flan_T5_Small_Subjects_df = pd.DataFrame(zipped_summaries, columns = ['Original_subjects', 'Flan_T5s_subjects', 'Finetuned_Flan_T5s_subjects'])
Flan_T5_Small_Subjects_df

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
# from datasets import Dataset


# # Initialize tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# T5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [ ]:
def tokenize_function(example):
    start_prompt = 'Generate the subject line for the following email.\n\n'
    end_prompt = '\n\Subject: '
    prompt = [start_prompt + email + end_prompt for email in example["cleaned_emails"]]
    # Tokenize the input_ids - do not flatten
    example['input_ids'] = [tokenizer(p, padding="max_length", truncation=True, return_tensors="pt").input_ids[0] for p in prompt]
    # Tokenize the labels - do not flatten
    example['labels'] = [tokenizer(s, padding="max_length", truncation=True, return_tensors="pt").input_ids[0] for s in example["subject"]]

    return example

In [ ]:
# output_dir = f'./email-Subject-training-{str(int(time.time()))}'

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     learning_rate=1e-5,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_steps=30,
#     evaluation_strategy='epoch',
#     per_device_train_batch_size=4,  # Reduced batch size
#     # max_steps=1
# )

# # training_args = TrainingArguments(
# #     output_dir='./results',
# #     evaluation_strategy='epoch',
# #     learning_rate=1e-5,
# #     per_device_train_batch_size=4,
# #     per_device_eval_batch_size=4,
# #     num_train_epochs=3,
# #     weight_decay=0.01,
# # )

# trainer = Trainer(
#     model=flan_t5_base_original_model,
#     args=training_args,
#     train_dataset=tokenized_train_datasets,
#     eval_dataset=tokenized_test_datasets
# )

# trainer.train()

In [ ]:
# # Generate subject lines for the test set
# def generate_subject_line(email_body):
#     inputs = tokenizer("generate subject line: " + email_body, return_tensors="pt", max_length=512, truncation=True)
#     # Move inputs to the same device as the model
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     outputs = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# test_df['generated_subject_line'] = test_df['cleaned_emails'].apply(generate_subject_line)

# Generate subject lines for the test set
def generate_subject_line(email_body):
    inputs = tokenizer("generate subject line: " + email_body, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # Access input_ids as a key in the dictionary
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_df['generated_subject_line'] = test_df['cleaned_emails'].apply(generate_subject_line)

# **t5-small Evaluation**

In [ ]:
!pip install nltk
import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [ ]:
# Download the 'punkt' resource
nltk.download('punkt')
nltk.download('wordnet') # Download WordNet

from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # Tokenize the input for METEOR
    true_subject_tokens = nltk.word_tokenize(true_subject)
    generated_subject_tokens = nltk.word_tokenize(generated_subject)

    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject_tokens], generated_subject_tokens) # Pass tokenized input

    # SACREBLEU - Use import sacrebleu directly
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu_score # Return the score, not the module

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject'], row['generated_subject_line']), axis=1))

# Calculate average scores
average_rougeL = test_df['rougeL'].mean()
average_meteor = test_df['meteor'].mean()
average_sacrebleu = test_df['sacrebleu'].mean() # Now you should be able to calculate the mean

print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')

# **Pretrained('facebook/bart-base')**

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
# # Convert DataFrame to Dataset
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

# Initialize tokenizer and model
facebook_bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
facebook_bart_base_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
print("For Model : facebook_bart_base_model \n",print_number_of_trainable_model_parameters(facebook_bart_base_model))

In [ ]:
prompt = f"""
Generate the subject line for the following email.

Email:
{email}

Subject:
"""

inputs = facebook_bart_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(facebook_bart_base_model.device) for k, v in inputs.items()}

outputs = facebook_bart_base_model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FB BART MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

In [ ]:

# Tokenization
def Tokenization_preprocess_data(examples):
    inputs = [prompt + email for email in examples['cleaned_emails']]
    model_inputs = facebook_bart_tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Use facebook_bart_tokenizer as the target tokenizer
    with facebook_bart_tokenizer.as_target_tokenizer():
        labels = facebook_bart_tokenizer(examples['subject'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(Tokenization_preprocess_data, batched=True)
test_dataset = test_dataset.map(Tokenization_preprocess_data, batched=True)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )


training_args = TrainingArguments(
    output_dir='./results',        # Directory to save the model checkpoints
    evaluation_strategy='steps',
    save_steps=500,               # Save checkpoint every 1000 steps
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
)

# Define Trainer
facebook_bart_base_model_trainer = Trainer(
    model=facebook_bart_base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
facebook_bart_base_model_trainer.train()


In [ ]:
# Save the final model checkpoint
facebook_bart_base_model.save_pretrained('./results/FB_Bart_model')
facebook_bart_tokenizer.save_pretrained('./results/FB_Bart_model')

# Define and save the generation configuration
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

generation_config.save_pretrained('./results/FB_Bart_model')

In [ ]:
cd /content

In [ ]:
# Define paths
logs_folder = '/content/logs'  # Adjust to your source folder
logs_git_folder = './content/qM-AI-L/email-subject/model-tuning/logs'  # Adjust to your repository folder

# Copy the folder
shutil.copytree(logs_folder, logs_git_folder)

In [ ]:
# Define paths
results_folder = '/content/results'  # Adjust to your source folder
results_git_folder = './content/qM-AI-L/email-subject/model-tuning/results'  # Adjust to your repository folder

# Copy the folder
shutil.copytree(results_folder, results_git_folder)

In [ ]:
# Change directory to the cloned repository
os.chdir('./content/qM-AI-L')  # Adjust to your repository folder

# Configure Git (only needed once)
!git config --global user.email "mlreddy3082@gmail.com"
!git config --global user.name "Mlr9459"

# Add, commit, and push changes
!git add /content/qM-AI-L/email-subject/
!git commit -m "Add training outputs: model checkpoints, tokenizer, and generation configuration"
!git push origin master

**Evaluate the Model Quantitatively (with ROUGE Metric)
The ROUGE metric) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.**

In [ ]:
tokenized_train_datasets, tokenized_test_datasets

In [ ]:
tokenized_train_datasets[0:10]['cleaned_emails']

In [ ]:
emails_for_score = tokenized_test_datasets[0:10]['cleaned_emails']
human_annoted_subjects = tokenized_test_datasets[0:10]['subject']


In [ ]:
facebook_bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
facebook_bart_base_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
model_directory = '/content/results/FB_Bart_model'
facebook_bart_tokenizer_trained = BartTokenizer.from_pretrained(model_directory)
facebook_bart_base_model_trained = BartForConditionalGeneration.from_pretrained(model_directory)

In [ ]:
facebook_bart_subjects = []
trained_facebook_bart_subjects = []

for _, email in enumerate(emails_for_score):
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """
    # Tokenize the input and move to the same device as the model
    inputs = facebook_bart_tokenizer(prompt, return_tensors='pt').to(facebook_bart_base_model.device)

    # Add the decoder_start_token_id to the generation configuration
    generation_config = GenerationConfig(
    max_new_tokens=200,
    decoder_start_token_id=facebook_bart_tokenizer.bos_token_id  # Add this line
    )



    # Generate subject using the original model
    original_model_outputs = facebook_bart_base_model.generate(**inputs, generation_config=generation_config)
    original_model_subject_output = facebook_bart_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    facebook_bart_subjects.append(original_model_subject_output)

    trained_facebook_bart_outputs = facebook_bart_base_model_trained.generate(**inputs, generation_config=generation_config)
    instruct_model_text_output = facebook_bart_tokenizer_trained.decode(trained_facebook_bart_outputs[0], skip_special_tokens=True)
    trained_facebook_bart_subjects.append(instruct_model_text_output)

zipped_summaries = list(zip(human_annoted_subjects, facebook_bart_subjects, trained_facebook_bart_subjects))

df = pd.DataFrame(zipped_summaries, columns = ['human_annoted_subjects', 'facebook_bart_subjects', 'trained_facebook_bart_subjects'])
df

In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer
from rouge_score.scoring import BootstrapAggregator



In [ ]:
# Calculate ROUGE scores for each pair of prediction and reference
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
aggregator = BootstrapAggregator() # Initialize aggregator

for prediction, reference in zip(facebook_bart_subjects, human_annoted_subjects[0:len(facebook_bart_subjects)]):
    score = scorer.score(reference, prediction)
    aggregator.add_scores(score) # Add scores to aggregator

# Aggregate the scores
original_model_results = aggregator.aggregate()

print(original_model_results)

In [ ]:
original_model_results = rouge.compute(
    predictions=facebook_bart_subjects,
    references=human_annoted_subjects[0:len(facebook_bart_subjects)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=trained_facebook_bart_subjects,
    references=human_annoted_subjects[0:len(trained_facebook_bart_subjects)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

In [ ]:
# Save the model locally
model_save_path = "./facebook_bart_base"
facebook_bart_base_model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

In [ ]:
# Generate subject lines for the test set
def generate_subject_line(email_body):
    inputs = tokenizer("generate subject line: " + email_body, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs to the GPU if available
    inputs = {k: v.to(facebook_bart_base_model.device) for k, v in inputs.items()}
    # Access input_ids from the dictionary
    outputs = facebook_bart_base_model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_df['generated_subject_line'] = test_df['body'].apply(generate_subject_line)

In [ ]:
test_df

In [ ]:

import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # Tokenize the input for METEOR
    true_subject_tokens = nltk.word_tokenize(true_subject)
    generated_subject_tokens = nltk.word_tokenize(generated_subject)

    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject_tokens], generated_subject_tokens) # Pass tokenized input

    # SACREBLEU - Use import sacrebleu directly
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu_score # Return the score, not the module

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject'], row['generated_subject_line']), axis=1))

In [ ]:

# Calculate average scores
average_rougeL = test_df['rougeL'].mean()
average_meteor = test_df['meteor'].mean()
average_sacrebleu = test_df['sacrebleu'].mean() # Now you should be able to calculate the mean

print(f'Average facebook_bart_base_model ROUGE-L: {average_rougeL:.4f}')
print(f'Average facebook_bart_base_model METEOR: {average_meteor:.4f}')
print(f'Average facebook_bart_base_model SACREBLEU: {average_sacrebleu:.4f}')

In [ ]:
import shutil
from google.colab import files

# Replace 'directory_name' with the name of your directory
shutil.make_archive('/content/facebook_bart_base_model', 'zip', 'facebook_bart_base_model')

In [ ]:
# files.download('facebook_bart_base_model.zip')

# **t5-base**

In [ ]:
# Define tokenizer and model t5-base
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")


In [ ]:
dataset = Dataset.from_pandas(pivot_df)

In [ ]:
dataset

In [ ]:
def preprocess_function(pivot_df):
    inputs = pivot_df['cleaned_emails']
    targets = pivot_df['subject']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
import shutil
from google.colab import files

# Replace 'directory_name' with the name of your directory
# shutil.make_archive('/content/fine_tuned_t5', 'zip', 'fine_tuned_t5')

# Download the zipped directory


In [ ]:
shutil.make_archive('/content/results', 'zip', 'FB_Bart_model_results')
shutil.make_archive('/content/logs', 'zip', 'FB_Bart_model_logs')


In [ ]:
# files.download('fine_tuned_t5.zip')

In [ ]:
tokenized_dataset

In [ ]:
# tokenized_dataset['attention_mask']
# tokenized_dataset['input_ids']

In [ ]:
# Split the dataset into train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    # predict_with_generate=True,
    logging_dir="./logs",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

In [ ]:
model.save_pretrained('fine_tuned_t5')
tokenizer.save_pretrained('fine_tuned_t5')

In [ ]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available. \nUsing GPU")
    device = torch.device('cuda')
else:
    print("GPU is not available. \nUsing CPU")
    device = torch.device('cpu')

In [ ]:
def generate_summary(text):
    inputs = tokenizer.encode_plus(text, return_tensors='pt', max_length=512, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    summary_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary



In [ ]:
# Example inference
example_text = pivot_df['body'][3]
print("Email Body:", example_text)
print("\n\n Generated Subject:", generate_summary(example_text))
print("\n\n Actual Subject:", pivot_df['subject'][3])

In [ ]:
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject], generated_subject)

    # SACREBLEU
    sacre_bleu = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject_line'], row['generated_subject_line']), axis=1))

# Calculate average scores
average_rougeL = test_df['rougeL'].mean()
average_meteor = test_df['meteor'].mean()
average_sacrebleu = test_df['sacrebleu'].mean()

print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')

In [ ]:
import re

def extract_sentences(text):
  """
  This function extracts sentences from text without introducing control characters.

  Args:
      text (str): The text to be processed.

  Returns:
      list: A list of extracted sentences.
  """
  # Split text based on sentence boundaries (adjust pattern as needed)
  sentences = re.split(r"(?<!\w)\.(?!\w)", text)

  # Remove empty elements and leading/trailing whitespace
  sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

  return sentences

# Example usage
text = "This is an example text. It contains multiple sentences.  \nThere are also newlines and extra spaces."
extracted_sentences = extract_sentences(pivot_df['body'][0])
print(extracted_sentences)


In [ ]:
pivot_df['body'][0]

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(pivot_df['body'][0], max_length=15, min_length=2, do_sample=False)
print(summary[0]['summary_text'])

In [ ]:
pivot_df['body'][0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("Chirayu/subject-generator-t5-base")
tokenizer = AutoTokenizer.from_pretrained("Chirayu/subject-generator-t5-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def get_subject(content, num_beams=5,max_length=512, repetition_penalty=2.5, length_penalty=1, early_stopping=True,top_p=.95, top_k=50, num_return_sequences=3):

  text =  "title: " + content + " </s>"

  input_ids = tokenizer.encode(
    text, return_tensors="pt", add_special_tokens=True
  )

  input_ids = input_ids.to(device)
  generated_ids = model.generate(
      input_ids=input_ids,

      num_beams=num_beams,
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      length_penalty=length_penalty,
      early_stopping=early_stopping,
      top_p=top_p,
      top_k=top_k,
      num_return_sequences=num_return_sequences,
  )
  subjects = [tokenizer.decode(generated_id,skip_special_tokens=True,clean_up_tokenization_spaces=True,) for generated_id in generated_ids]
  return subjects

In [ ]:
x = get_subject(pivot_df['body'][0])
x

In [ ]:
pivot_df['body'][0]

In [ ]:
pivot_df['body'][1]

In [ ]:
import textwrap

In [ ]:
def text_summarizer(email_text):

    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    inputs = tokenizer.encode("summarize: " + email_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=100, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    formatted_summary = "\n".join(textwrap.wrap(summary, width=80))
    return formatted_summary

summary = text_summarizer(pivot_df['body'][0])
summary

In [ ]:
def extract_sentences(text):
    """
    Extract sentences from the text and remove newline characters.

    Parameters:
    text (str): The text to process.

    Returns:
    list: A list of sentences without newline characters.
    """
    # Replace newline characters with spaces
    text = text.replace('\n', '')
    text = text.replace('!', '.').replace('?', '.').replace(';', '.')

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    return sentences

# Example usage
sample_text = "Hello, world!\nThis is a test.\nLet's replace, commas, and newlines."
sentences = extract_sentences(sample_text)
for sentence in sentences:
    print(sentence)

In [ ]:
summary = text_summarizer(pivot_df['body'][0])
summary

In [ ]:
pivot_df['body'][0]

In [ ]:
email_len = [len(pivot_df['dialogue'].split()) for x in samsum['train']]
sub_len = [len(pivot_df['summary'].split()) for x in samsum['train']]

dialogue_len = [len(x['dialogue'].split()) for x in samsum['train']]
summary_len = [len(x['summary'].split()) for x in samsum['train']]



In [ ]:
describe_df(pivot_df)

In [ ]:
nan_rows

In [ ]:
from langchain.preprocessing import Tokenizer, Lowercaser, PunctuationRemover, StopwordRemover
from langchain.stemming import Stemmer
from langchain import Pipeline

In [ ]:
len(email_docs)*3

In [ ]:
emails_df['content'][0]

In [ ]:
emails_df['source'][0]

In [ ]:
extract_file_name(emails_df['source'][0])